In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import pytorch_lightning as pl
import os
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import pandas as pd
from sklearn.model_selection import train_test_split
from torchmetrics.functional import accuracy
from sklearn import metrics
import torch
import csv
from sklearn import metrics
from matplotlib import pyplot as plt
import math
import copy
import random

torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)


tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# done cause every time I change data_preprocessing.py it didn't affect
import importlib

# import outsiders
sys.path.append('..')
import data_preprocessing
import ECG
importlib.reload(data_preprocessing)
importlib.reload(ECG)


/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-02 21:58:58.594212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 21:59:04.213466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-02 21:59:04.213839: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dy

<module 'ECG' from '../ECG.py'>

In [2]:
swell_data = data_preprocessing.load_data(os.path.join("../swell_dataset","swell_dict_index_6.npy"))
swell_data = data_preprocessing.swell_prepare_for_10fold(swell_data)
before=copy.copy(swell_data)
#fix the range of the labels to init from value of zero
swell_data[:,1:4]-=1

In [3]:
def train_test_indexes_var2(ratio, objects):
    # Split objects into train and test sets
    num_test = int(len(set(objects)))-int(len(set(objects)) * ratio)
    obj_list = list(set(objects))
    random.shuffle(obj_list)
    test_objs = obj_list[:num_test]
    train_objs = obj_list[num_test:]

    train_idxs = []
    test_idxs = []

    # Assign each object's samples to the appropriate set
    for obj in train_objs:
        obj_idxs = [i for i, o in enumerate(objects) if o == obj]
        train_idxs.extend(obj_idxs)

    for obj in test_objs:
        obj_idxs = [i for i, o in enumerate(objects) if o == obj]
        test_idxs.extend(obj_idxs)

    # Shuffle the final train and test sets
    random.shuffle(train_idxs)
    random.shuffle(test_idxs)

    return (train_idxs), (test_idxs)
# check train_test_indexes_var2()
# not for each runnig acieve exaclty 9:1 train:test
ratio = 0.9
objects=swell_data[:,0]
print("tot len is ", len(objects))
print("trin len is ",ratio*len(objects))
print("test len is ",len(objects)-ratio*len(objects))
i=train_test_indexes_var2(ratio, objects)
#print("train idx ",i[0])
#print("test idx ",i[1])
print("train obj:")
tr=[]
for j in i[0]:
  #print(objects[int(j)])
  tr.append(objects[int(j)])
tr_u=set(tr)

print("test obj:")
te=[]
for j in i[1]:
  #print(objects[int(j)])
  te.append(objects[int(j)])
te_u=set(te)

print(tr_u,len(tr))
print(te_u,len(te))
splits=i
np.save("./splits_B",splits)

tot len is  14595
trin len is  13135.5
test len is  1459.5
train obj:
test obj:
{2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0, 10.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0} 12567
{1.0, 18.0, 25.0} 2028


/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [4]:
x_tr=[]
y_tr=[]
for i in splits[0]:
    x_tr.append(swell_data[i,4:])
    y_tr.append(swell_data[i,:4])
x_train=np.array(x_tr)
y_train=np.array(y_tr)

x_te=[]
y_te=[]
for i in splits[1]:
    x_te.append(swell_data[i,4:])
    y_te.append(swell_data[i,:4])
x_test=np.array(x_te)
y_test=np.array(y_te)

In [16]:
(x_train).shape
print(type(x_train))

<class 'numpy.ndarray'>


In [5]:
x_train = torch.from_numpy(x_train).float().unsqueeze(1)
#y_train = torch.from_numpy(y_train).long()
y_train = torch.tensor(y_train).long()
x_test = torch.from_numpy(x_test).float().unsqueeze(1)
#y_test = torch.from_numpy(y_test).long()
y_test = torch.tensor(y_test).long()
tensor_dataset = torch.utils.data.TensorDataset(x_train, y_train)
tensor_dataset_test = torch.utils.data.TensorDataset(x_test, y_test)
from sklearn import metrics

train_loader = DataLoader(tensor_dataset, batch_size=128) # for debug set num_workers=0
test_loader = DataLoader(tensor_dataset_test,batch_size=128) # for debug set num_workers=0

In [6]:
# model
model =ECG.EmotionRec()
# training
trainer = pl.Trainer(gpus = 1 , max_epochs = 100)
torch.save(model.state_dict(), './model_light_100_B.pt')
trainer.fit(model, train_loader,test_loader)

trainer = pl.Trainer(gpus = 1 , max_epochs = 150)
torch.save(model.state_dict(), './model_light_250_B.pt')
trainer.fit(model, train_loader,test_loader)

/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | cnn     | Sequential | 329 K 
1 | stress  | Sequential | 330 K 
2 | arousal | Sequential | 333 K 
3 | valence | Sequential | 333 K 
---------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.304     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,
/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 99: 100%|██████████| 115/115 [00:03<00:00, 33.14it/s, loss=0.258, v_num=5, train_stress_acc=1.000, train_arousal_acc=1.000, train_valence_acc=1.000, test_loss_stress=3.320, test_acc_stress=0.449, test_loss_arousal=16.60, test_acc_arousal=0.0246, test_loss_valence=10.80, test_acc_valence=0.0355]

...

/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | cnn     | Sequential | 329 K 
1 | stress  | Sequential | 330 K 
2 | arousal | Sequential | 333 K 
3 | valence | Sequential | 333 K 
---------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.304     Total estimated model params size (MB)


/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 149: 100%|██████████| 115/115 [00:03<00:00, 33.68it/s, loss=0.076, v_num=6, train_stress_acc=1.000, train_arousal_acc=1.000, train_valence_acc=1.000, test_loss_stress=5.200, test_acc_stress=0.401, test_loss_arousal=19.00, test_acc_arousal=0.0177, test_loss_valence=14.20, test_acc_valence=0.0538]

`Trainer.fit` stopped: `max_epochs=150` reached.


Epoch 149: 100%|██████████| 115/115 [00:03<00:00, 32.14it/s, loss=0.076, v_num=6, train_stress_acc=1.000, train_arousal_acc=1.000, train_valence_acc=1.000, test_loss_stress=5.200, test_acc_stress=0.401, test_loss_arousal=19.00, test_acc_arousal=0.0177, test_loss_valence=14.20, test_acc_valence=0.0538]
